<a href="https://colab.research.google.com/github/uqmshawn/uqmshawn-4-7-1-8-4-5-3-0-r/blob/main/47184530.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from google.colab import drive
import zipfile

In [6]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mount Google Drive
drive.mount('/content/drive')

class BrainSlicesDataset(Dataset):
    def __init__(self, image_slices):
        self.image_slices = image_slices

    def __len__(self):
        return len(self.image_slices)

    def __getitem__(self, idx):
        image = self.image_slices[idx]

        # Ensure the image has a channel dimension
        if len(image.shape) == 2:  # If the image is of shape [H, W]
            image = torch.unsqueeze(image, 0)  # Convert it to [1, H, W]

        return image

def get_image_slices():
    zip_path = "/content/drive/MyDrive/Colab_Notebooks_Course/image_process/A3/testgans/GAN_Dataset.zip"
    extraction_path = "/content/GAN_Dataset"
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    parent_dir = "/content/GAN_Dataset"
    train_path = os.path.join(parent_dir, "keras_png_slices_train")
    test_path = os.path.join(parent_dir, "keras_png_slices_test")
    val_path = os.path.join(parent_dir, "keras_png_slices_validate")

    def load_images_from_folder(folder_path):
            images = []
            for filename in os.listdir(folder_path):
                img = Image.open(os.path.join(folder_path, filename)).convert('L').resize((128, 128))
                if img is not None:
                    images.append(torch.tensor(np.array(img, dtype=np.float32)))
            return torch.stack(images)

    train_images = load_images_from_folder(train_path)
    test_images = load_images_from_folder(test_path)
    validate_images = load_images_from_folder(val_path)

    # Print statements to understand the data
    print(f"Total train images: {len(train_images)}")
    print(f"Shape of a single train image: {train_images[0].shape}")
    print(f"Total test images: {len(test_images)}")
    print(f"Shape of a single test image: {test_images[0].shape}")
    print(f"Total validation images: {len(validate_images)}")
    print(f"Shape of a single validation image: {validate_images[0].shape}")

    return train_images, test_images, validate_images

# Call the function to see the print outputs
get_image_slices()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total train images: 9664
Shape of a single train image: torch.Size([128, 128])
Total test images: 544
Shape of a single test image: torch.Size([128, 128])
Total validation images: 1120
Shape of a single validation image: torch.Size([128, 128])


(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [ ]:
# Model Definitions: VectorQuantizer, Encoder, Decoder, VQVAE, PixelCNN, etc.

# VectorQuantizer Layer
# Encoder
# Decoder
# VQVAE
# VQVAETrainer
# PixelConvLayer & PixelCNN

In [ ]:
# Training Functions: train_vqvae, train_pixelcnn, etc.

# train_vqvae
# train_pixelcn

In [ ]:
import matplotlib.pyplot as plt

def visualize_reconstructions(originals, reconstructions, num_samples=3):
    # ... [Code for visualizing reconstructions]

def visualize_samples(samples, num_samples=3):
    # ... [Code for visualizing samples]

In [ ]:
def main():
    # ... [Main function code]

if __name__ == "__main__":
    main()